In [1]:
import pandas as pd

In [2]:
query = "SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` WHERE date BETWEEN '20170101' AND '20170131'"

In [3]:
df = pd.read_gbq(query=query, project_id='intense-elysium-406619', dialect='standard')
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,3,1485003922,1485003922,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Europe', 'subContinent': 'Easte...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",6786265513681298458,None,None,Direct,Not Socially Engaged
1,<NA>,1,1485062353,1485062353,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 21, 'minu...",0139439830743342456,None,None,Organic Search,Not Socially Engaged
2,<NA>,1,1485060552,1485060552,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 20, 'minu...",3783938919763923299,None,None,Referral,Not Socially Engaged
3,<NA>,1,1485017601,1485017601,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",0693204770361960529,None,None,Organic Search,Not Socially Engaged
4,<NA>,8,1485025469,1485025469,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",5696667685436779495,None,None,Direct,Not Socially Engaged


In [4]:
df.shape

(64694, 16)

In [5]:
df['fullVisitorId'] = df['fullVisitorId'].astype(str)

In [6]:
produtos_sessao = []
precos_sessao = []

for linha in df['hits']:
    produtos_hit = []
    precos_hit = []
    
    for hit in linha:
        for produto in hit['product']:
            produtos_hit.append(produto['productSKU'])
            precos_hit.append(produto['productPrice'])
    
    produtos_sessao.append(produtos_hit)
    precos_sessao.append(sum(precos_hit))

In [7]:
produtos_sessao[:1]

[['GGOEGBRB013899', 'GGOEGBRB013899']]

In [8]:
precos_sessao[:5]

[199980000, 451760000, 469950000, 792830000, 221860000]

In [9]:
produtos_series = pd.Series(produtos_sessao)
produtos_series

0                         [GGOEGBRB013899, GGOEGBRB013899]
1        [GGOEGAAX0279, GGOEGAAX0291, GGOEGAAX0278, GGO...
2        [GGOEGBRJ037299, GGOEGBRA037499, GGOEGBRB01389...
3        [GGOEGCLB020832, GGOEGCMB020932, GGOEGBFC01879...
4        [GGOEAKDH019899, GGOEGODR017799, GGOEGOFH02029...
                               ...                        
64689                                                   []
64690                                                   []
64691                                                   []
64692                                                   []
64693                                                   []
Length: 64694, dtype: object

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

In [11]:
mlb = MultiLabelBinarizer()

In [12]:
df_produtos = pd.DataFrame(mlb.fit_transform(produtos_series), columns=mlb.classes_, index=df.index)
df_produtos.head()

,10 52048,10 52147,10 52211,10 52213,10 52223,10 52225,10 52230,10 52231,10 52232,10 52233,...,GGOEYFKQ020699,GGOEYHPA003510,GGOEYHPA003610,GGOEYHPB072210,GGOEYOBR078599,GGOEYOCR077399,GGOEYOCR077799,GGOEYOCR078099,GGOEYOLR018699,GGOEYOLR080599
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [13]:
df = df.join(df_produtos)
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,...,GGOEYFKQ020699,GGOEYHPA003510,GGOEYHPA003610,GGOEYHPB072210,GGOEYOBR078599,GGOEYOCR077399,GGOEYOCR077799,GGOEYOCR078099,GGOEYOLR018699,GGOEYOLR080599
0,<NA>,3,1485003922,1485003922,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Europe', 'subContinent': 'Easte...","[{'index': 4, 'value': 'EMEA'}]",...,0,0,0,0,0,0,0,0,0,0
1,<NA>,1,1485062353,1485062353,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...",[],...,0,0,0,0,0,0,0,0,0,0
2,<NA>,1,1485060552,1485060552,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]",...,0,0,0,0,0,0,0,0,0,0
3,<NA>,1,1485017601,1485017601,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]",...,0,0,0,0,0,0,0,0,0,0
4,<NA>,8,1485025469,1485025469,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]",...,1,0,0,0,0,0,0,0,0,0


In [14]:
df['preco'] = precos_sessao
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,...,GGOEYHPA003510,GGOEYHPA003610,GGOEYHPB072210,GGOEYOBR078599,GGOEYOCR077399,GGOEYOCR077799,GGOEYOCR078099,GGOEYOLR018699,GGOEYOLR080599,preco
0,<NA>,3,1485003922,1485003922,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Europe', 'subContinent': 'Easte...","[{'index': 4, 'value': 'EMEA'}]",...,0,0,0,0,0,0,0,0,0,199980000
1,<NA>,1,1485062353,1485062353,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...",[],...,0,0,0,0,0,0,0,0,0,451760000
2,<NA>,1,1485060552,1485060552,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]",...,0,0,0,0,0,0,0,0,0,469950000
3,<NA>,1,1485017601,1485017601,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]",...,0,0,0,0,0,0,0,0,0,792830000
4,<NA>,8,1485025469,1485025469,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]",...,0,0,0,0,0,0,0,0,0,221860000


In [15]:
df_visitas = df.drop(columns=['hits', 'customDimensions'], axis=1)
df_visitas.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,fullVisitorId,...,GGOEYHPA003510,GGOEYHPA003610,GGOEYHPB072210,GGOEYOBR078599,GGOEYOCR077399,GGOEYOCR077799,GGOEYOCR078099,GGOEYOLR018699,GGOEYOLR080599,preco
0,<NA>,3,1485003922,1485003922,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Europe', 'subContinent': 'Easte...",6786265513681298458,...,0,0,0,0,0,0,0,0,0,199980000
1,<NA>,1,1485062353,1485062353,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...",0139439830743342456,...,0,0,0,0,0,0,0,0,0,451760000
2,<NA>,1,1485060552,1485060552,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...",3783938919763923299,...,0,0,0,0,0,0,0,0,0,469950000
3,<NA>,1,1485017601,1485017601,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...",0693204770361960529,...,0,0,0,0,0,0,0,0,0,792830000
4,<NA>,8,1485025469,1485025469,20170121,"{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...",5696667685436779495,...,0,0,0,0,0,0,0,0,0,221860000


In [16]:
dicionarios = ['device', 'trafficSource', 'geoNetwork', 'totals']

In [17]:
import json

for coluna in dicionarios:
  df_visitas = df_visitas.join(
    pd.DataFrame([json.loads(json.dumps(linha))
      for linha in df_visitas[coluna]]) , rsuffix=('_' + coluna) )
df_visitas.drop( dicionarios, axis=1, inplace=True)

totals = df.totals[0].keys()
totals = list(totals)

for coluna in totals:
  df_visitas[coluna] = pd.to_numeric(df_visitas[coluna])

df_visitas.drop('adwordsClickInfo', axis=1, inplace=True)
coluna_na = []

for coluna in df_visitas.columns:
  print(str(coluna) + ': ' + str(len(df_visitas[coluna].unique())))
  if len(df_visitas[coluna].unique()) == 1:
    coluna_na.append(coluna)
df_visitas.drop( coluna_na, axis=1, inplace=True)

df_visitas.head()

visitorId: 1
visitNumber: 202
visitId: 63585
visitStartTime: 63640
date: 31
fullVisitorId: 53041
userId: 1
clientId: 1
channelGrouping: 8
socialEngagementType: 1
10 52048: 2
10 52147: 2
10 52211: 2
10 52213: 2
10 52223: 2
10 52225: 2
10 52230: 2
10 52231: 2
10 52232: 2
10 52233: 2
10 52234: 2
10 52237: 2
10 52238: 2
10 52242: 2
10 52246: 2
10 52250: 2
10 52251: 2
9180748: 2
9180749: 2
9180750: 2
9180751: 2
9180752: 2
9180754: 2
9180755: 2
9180756: 2
9180757: 2
9180758: 2
9180759: 2
9180760: 2
9180761: 2
9180764: 2
9180766: 2
9180767: 2
9180769: 2
9180770: 2
9180771: 2
9180778: 2
9180779: 2
9180781: 2
9180782: 2
9180785: 2
9180791: 2
9180792: 2
9180793: 2
9180797: 2
9180801: 2
9180802: 2
9180804: 2
9180806: 2
9180808: 2
9180809: 2
9180811: 2
9180812: 2
9180814: 2
9180815: 2
9180816: 2
9180817: 2
9180818: 2
9180819: 2
9180820: 2
9180822: 2
9180824: 2
9180833: 2
9180834: 2
9180837: 2
9180838: 2
9180840: 2
9180841: 2
9180844: 2
9180850: 2
9180862: 2
9180863: 2
9180875: 2
9180905: 2
9181019

,visitNumber,visitId,visitStartTime,date,fullVisitorId,channelGrouping,10 52048,10 52147,10 52211,10 52213,...,city,networkDomain,hits,pageviews,timeOnSite,bounces,transactions,transactionRevenue,newVisits,totalTransactionRevenue
0,3,1485003922,1485003922,20170121,6786265513681298458,Direct,0,0,0,0,...,Warsaw,swps.edu.pl,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1485062353,1485062353,20170121,0139439830743342456,Organic Search,0,0,0,0,...,not available in demo dataset,wlink.com.np,2,1.0,126.0,NaN,NaN,NaN,1.0,NaN
2,1,1485060552,1485060552,20170121,3783938919763923299,Referral,0,0,0,0,...,not available in demo dataset,comcast.net,2,2.0,7.0,NaN,NaN,NaN,1.0,NaN
3,1,1485017601,1485017601,20170121,0693204770361960529,Organic Search,0,0,0,0,...,not available in demo dataset,(not set),2,2.0,37.0,NaN,NaN,NaN,1.0,NaN
4,8,1485025469,1485025469,20170121,5696667685436779495,Direct,0,0,0,0,...,not available in demo dataset,(not set),2,2.0,8.0,NaN,NaN,NaN,NaN,NaN


In [18]:
df_visitas = df_visitas.drop(columns=['transactionRevenue', 'transactions'], axis=1)

In [19]:
df_visitas['totalTransactionRevenue'] = df_visitas['totalTransactionRevenue'] / 1000000

In [23]:
quantitativas = list(set(totals) - set(coluna_na) - set(['transactionRevenue', 'transactions']))

df_visitas_totals = df_visitas.groupby('fullVisitorId', as_index=False)[quantitativas].sum()

df_visitas_ultima = df_visitas.groupby('fullVisitorId', as_index=False)
df_visitas_ultima = df_visitas_ultima['visitNumber'].max()

df_usuarios_visitas_unicos = df_visitas.drop_duplicates(subset=['fullVisitorId', 'visitNumber'])

df_usuarios = pd.merge(df_visitas_ultima, df_usuarios_visitas_unicos, left_on=['fullVisitorId', 'visitNumber'], right_on=['fullVisitorId', 'visitNumber'],
                    how='left')

df_visitas_primeira = df_visitas.groupby('fullVisitorId', as_index=False)['visitNumber'].min()
df_visitas_primeira.set_index('fullVisitorId', inplace=True)

df_usuarios = df_usuarios.join(df_visitas_primeira, how='left', on='fullVisitorId', rsuffix='_primeira')
df_usuarios = pd.merge(df_usuarios, df_usuarios_visitas_unicos, left_on=['fullVisitorId', 'visitNumber_primeira'],
                    right_on=['fullVisitorId', 'visitNumber'], how='left', suffixes=('_ultima', '_primeira'))

df_usuarios = pd.merge(df_usuarios, df_visitas_totals, left_on=['fullVisitorId'], right_on=['fullVisitorId'], how='left')

for i in quantitativas:
  df_usuarios.drop(i+'_primeira', axis=1, inplace=True)
  df_usuarios.drop(i+'_ultima', axis=1, inplace=True)

df_usuarios['tempo_visitas'] = df_usuarios['visitStartTime_ultima'] - df_usuarios['visitStartTime_primeira']

df_usuarios.head()

,fullVisitorId,visitNumber_ultima,visitId_ultima,visitStartTime_ultima,date_ultima,channelGrouping_ultima,10 52048_ultima,10 52147_ultima,10 52211_ultima,10 52213_ultima,...,metro_primeira,city_primeira,networkDomain_primeira,newVisits,bounces,timeOnSite,totalTransactionRevenue,hits,pageviews,tempo_visitas
0,0000176652019531658,1,1484597865,1484597865,20170116,Social,0,0,0,0,...,not available in demo dataset,not available in demo dataset,deltainfocom.com,1.0,1.0,0.0,0.0,1,1.0,0
1,0000543175149799625,1,1485675517,1485675517,20170128,Social,0,0,0,0,...,not available in demo dataset,not available in demo dataset,megaegg.ne.jp,1.0,1.0,0.0,0.0,1,1.0,0
2,0000613897283848534,1,1483595731,1483595731,20170104,Referral,0,0,0,0,...,Seattle-Tacoma WA,Kirkland,comcast.net,1.0,0.0,78.0,0.0,5,5.0,0
3,0000641787877778885,1,1483629067,1483629067,20170105,Organic Search,0,0,0,0,...,(not set),Paris,unknown.unknown,1.0,1.0,0.0,0.0,1,1.0,0
4,000072202462202136,1,1484513468,1484513468,20170115,Organic Search,0,0,0,0,...,not available in demo dataset,not available in demo dataset,virginm.net,1.0,1.0,0.0,0.0,1,1.0,0


In [25]:
ids = ['fullVisitorId', 'visitId_ultima', 'visitId_primeira']
df_usuarios.drop(columns=ids, axis=1, inplace=True)

X = df_usuarios.drop(columns=['totalTransactionRevenue'], axis=1)

y = df_usuarios['totalTransactionRevenue'].copy()
y.fillna(0, inplace=True)
y[y < 0] = 0

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
qualitativas = df_usuarios.dtypes[df_usuarios.dtypes == 'object'].keys()

le = LabelEncoder()

for coluna in qualitativas:
    X[coluna] = le.fit_transform(list(X[coluna].values.astype('str')))

In [28]:
from sklearn.model_selection import train_test_split

In [30]:
from sklearn.linear_model import LinearRegression

In [31]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [32]:
y_pred = lr.predict(X_test)

In [35]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [36]:
np.sqrt(mean_squared_error(y_test, y_pred))

53.604115801779905

In [38]:
from sklearn.ensemble import GradientBoostingRegressor

In [40]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)

GradientBoostingRegressor()

In [41]:
y_pred = gbr.predict(X_test)

In [42]:
np.sqrt(mean_squared_error(y_test, y_pred))

39.703472010857894

In [44]:
from sklearn.dummy import DummyRegressor

In [46]:
dr = DummyRegressor()
dr.fit(X_train, y_train)

DummyRegressor()

In [47]:
y_pred = dr.predict(X_test)

In [48]:
np.sqrt(mean_squared_error(y_test, y_pred))

43.006562211198855

In [49]:
resultados = pd.DataFrame()

resultados['revenue'] = y_test
resultados['predict'] = y_pred
resultados['erro'] = resultados['predict'] - resultados['revenue']

resultados.head()

,revenue,predict,erro
429,0.0,1.910492,1.910492
32524,0.0,1.910492,1.910492
44785,0.0,1.910492,1.910492
31742,0.0,1.910492,1.910492
33125,0.0,1.910492,1.910492


In [50]:
y_train.values[y_train <= 0] = 0
y_train.values[y_train > 0] = 1

y_test.values[y_test <= 0] = 0
y_test.values[y_test > 0] = 1

In [53]:
from sklearn.ensemble import GradientBoostingClassifier

In [54]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

GradientBoostingClassifier()

In [55]:
y_pred = gbc.predict(X_test)

In [56]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.10840386629573694

In [57]:
from sklearn.metrics import confusion_matrix

In [58]:
confusion_matrix(y_test, y_pred)

array([[15649,    61],
       [  126,    77]])

In [59]:
from sklearn.metrics import classification_report

In [60]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     15710
         1.0       0.56      0.38      0.45       203

    accuracy                           0.99     15913
   macro avg       0.77      0.69      0.72     15913
weighted avg       0.99      0.99      0.99     15913



In [61]:
X_train['revenue'] = y_train

In [62]:
df_usuarios_com_gastos = X_train[X_train['revenue'] > 0]
len(df_usuarios_com_gastos)

459

In [63]:
df_usuarios_sem_gastos = X_train[X_train['revenue'] <= 0]
len(df_usuarios_sem_gastos)

36669

In [64]:
from sklearn.utils import resample

In [65]:
df_usuarios_sem_gastos_ds = resample(df_usuarios_sem_gastos, replace=False, n_samples=30000, random_state=42)

In [68]:
X_train_ds = pd.concat([df_usuarios_sem_gastos_ds, df_usuarios_com_gastos])
len(X_train_ds)

30459

In [69]:
y_train_ds = X_train_ds['revenue']
X_train_ds = X_train_ds.drop(columns=['revenue'], axis=1)

gbc.fit(X_train_ds, y_train_ds)

GradientBoostingClassifier()

In [70]:
y_pred = gbc.predict(X_test)

In [71]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.106944797531706

In [72]:
confusion_matrix(y_test, y_pred)

array([[15645,    65],
       [  117,    86]])

In [73]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     15710
         1.0       0.57      0.42      0.49       203

    accuracy                           0.99     15913
   macro avg       0.78      0.71      0.74     15913
weighted avg       0.99      0.99      0.99     15913



In [75]:
from xgboost import XGBClassifier

In [80]:
X_train_ds = X_train_ds.loc[:, ~X_train_ds.columns.duplicated()]
X_test = X_test.loc[:, ~X_test.columns.duplicated()]

In [81]:
xgb = XGBClassifier(seed=42, n_estimators=300)
xgb.fit(X_train_ds, y_train_ds)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [83]:
y_pred = xgb.predict(X_test)

In [84]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.09901164312095938

In [85]:
confusion_matrix(y_test, y_pred)

array([[15650,    60],
       [   96,   107]])

In [86]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     15710
         1.0       0.64      0.53      0.58       203

    accuracy                           0.99     15913
   macro avg       0.82      0.76      0.79     15913
weighted avg       0.99      0.99      0.99     15913

